In [1]:
import os
import sys
src_path = sys.path[0].replace("scripts", "src")
if src_path not in sys.path:
    sys.path.append(src_path)
out_path = sys.path[0].replace("scripts", "output")

import numpy as np
from battery_model import *
from optimizer import *
from predictor import Predictor_tou_SDGE_DA, Predictor_tou_CAISO
from exp_manager import ExperimentManager
import copy

In [2]:
exp_suffix = "Mar2023"
exp_folder = os.path.join(out_path, "experiments", exp_suffix)
assert os.path.exists(exp_folder)

In [3]:
tou_pred = Predictor_tou_SDGE_DA(None)
tou = tou_pred.get_prediction(datetime(2019,1,1,0,0), 96, 0.25)

opt_params_sample = {
    "K": 96,
    "delta": 0.25,
    "S": 1, # number of scenarios (energy profiles). can omit, default 1
    "S_prob": None, 
    # if S != 1, params in "loads", "ev" should be list of length S
    ### loads
    "load_bld": 5 * np.sin(np.linspace(0, 2*np.pi, 96)) + 10,
    "load_pv": np.maximum(0, -np.linspace(-6,6,96)**2+20),
    "energy_price_buy": tou, # $/kWh
    "energy_price_sell": 0.6, # if None, sell is not allowed. can omit
    ### demand charge
    "dc_price": 0.6, # $/day. ref: 18 $/mon
    "dc_prev_max": None, # track p_grid_max in the same billing cycle. can omit
    "p_grid_max": "1.5",    # "1.5" means 1.5 x max(load_bld), a float, e.g., 1000, means 1000 kW, default: None
    ### battery (here I only consider one battery)
    """ Battery params will be provided by battery object """
    # "bat_capacity": None, # if none, capacity is optimized
    # "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    # "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    # "bat_price": 100, # $/kWh, ref: Tesla Powerwall
    # "bat_efficacy": 0.98, 
    # "bat_life_0": 3650, # days.
    # "bat_cycle_0": 5000, # cycles in lifetime
    "deg_model_opt": "throughput",  # valid values: "throughput", "Crate", "rainflow", "DOD"
    "deg_thres_opt": None,
    "deg_lambda_opt": None,
    "bat_soc_0": 0, # B0, BT have to be fractions (SoC indeed). 0.5 if omit FIXME: if 0.5, MSC can take advantage of this
    "bat_soc_K": None, # if None, default is the same as bat_soc_0
    ### EVs
    "ev_I": 20,
    "ev_ta": np.linspace(0, 48, 20), # ta, td can be floats
    "ev_td": np.linspace(36, 96, 20),
    "ev_e_init": np.array([0]*20),
    "ev_e_targ": np.array([10]*20),
    "ev_capacity": None, # can omit, default as e_targ (useful only when aloow discharge)
    "ev_p_max": 6.6,
    "ev_p_min": 0, # can omit, default as 0
    "ev_efficacy": 0.98,
    "ev_charge_rule": "optimal",
    "ev_charge_rule_default": "unif"
}


bat_params_sample ={
    "bat_capacity": None,
    "bat_p_max": 3, # i.e., capacity (kWh) / p_bat_max (kW) = 3 (h)
    "bat_p_min": 3, # can omit, then p_bat_min = p_bat_max
    "bat_price": 150, # $/kWh (old: 1000, ref: Tesla Powerwall)
    "bat_efficacy": 0.98, 
    "bat_life_0": 3650, # days.
    "bat_cycle_0": 3000, # cycles in lifetime
    # battery degradation params
    "deg_model": "throughput",  
        # valid values: "throughput", "Crate", "rainflow", "DOD"
    #   [1. degradation ~ high C-rate]
    "deg_Crate_thres": (0.25, 0.25, 0.25, 0.25),
    "deg_Crate_lambda": (0.8, 1, 1.5, 2),
    #   [2. degradation ~ large cycle depth]
    "deg_rainflow_thres": (0.2, 0.2, 0.2, 0.4),
    "deg_rainflow_lambda": (0.6, 1, 1.5, 1.8),
    #   [3. degradation ~ low SoE range]
    "deg_DOD_thres": (0.5, 0.2, 0.2, 0.1),
    "deg_DOD_lambda": (1.3, 1.15, 0.85, 0.6),
}

fn = "Mar19-Yi-base_rep_profiles.npy"
folder = os.path.join(exp_folder, "ref_data")
rep_profiles_params = np.load(os.path.join(folder,fn), allow_pickle=True).tolist()

In [4]:
class Size_TCO_rep_ExperimentManager(ExperimentManager):
    """
    Main variable: battery size (normalized to hr: to bld_load)
    """
    def run_one_trial(self, params, save_fn):
        
        # params: keys: "strategy", "B", "deg_model_opt", "deg_model", "ev_charge_rule"

        opt_params = copy.deepcopy(opt_params_sample)
        opt_params.update(rep_profiles_params)
        bat_params = copy.deepcopy(bat_params_sample)
        opt_params["deg_model_opt"] = params["deg_model_opt"]
        opt_params["ev_charge_rule"] = params["ev_charge_rule"]
        bat_params["deg_model"] = params["deg_model"]

        if params["B"] == "opt":
            bat_capacity = None
        else:
            K, delta = opt_params["K"][0], opt_params["delta"][0]
            prob = np.array(opt_params["S_prob"])

            prob /= prob.sum()  # normalize to sum 1 !!

            avg_load_S = np.array(opt_params["load_bld"]).sum(axis=1) * (24/K)
            avg_load = avg_load_S @ prob

            bat_capacity = params["B"]/24 * avg_load
        
        bat_params["bat_capacity"] = bat_capacity
        opt_params["bat_capacity"] = bat_capacity

        strategy = params["strategy"]

        b_sample = Battery_base(bat_params)
        opt = Battery_optimizer(battery=b_sample)

        if params["B"] == "opt":
            sol = opt.optimize_battery_size(opt_params, strategy=strategy, mute=True)
        else:
            sol = opt.get_control_sequence(opt_params, strategy=strategy, mute=True)
        df = sol.sol_summary()
        sol.save(save_fn=save_fn, save_path=self.save_path)
        
        
        stats = dict(df.T["All"])
        print("="*10, strategy, params["B"], stats["TCO"], "="*10)
        return stats

In [8]:
log_fn = os.path.join(exp_folder, "LOG-Size_TCO_rep.xlsx")
save_path = os.path.join(exp_folder, "EXP-size_tco_curve")
em = Size_TCO_rep_ExperimentManager(log_fn=log_fn, save_path=save_path, save=True, exp_prefix="Size_TCO_rep")
var_keys = ["strategy", "B", "deg_model_opt", "deg_model", "ev_charge_rule"]
em.run(keys=var_keys, num_trials=72)

========== MSC 1 238.96875096927832 ==========
Done, trial 54
========== MSC 2 236.27088519357164 ==========
Done, trial 55
========== MSC 3 234.54170312111034 ==========
Done, trial 56
========== MSC 4 234.00662060119762 ==========
Done, trial 57
========== MSC 5 234.30104674971653 ==========
Done, trial 58
========== MSC 6 235.77435508932308 ==========
Done, trial 59
========== MSC 7 238.1155486738201 ==========
Done, trial 60
========== MSC 8 240.69213442515237 ==========
Done, trial 61
========== MSC 9 243.02178044084206 ==========
Done, trial 62
========== MSC 10 245.35142645653178 ==========
Done, trial 63
========== MSC 11 247.68107247222147 ==========
Done, trial 64
========== MSC 12 250.0107184879112 ==========
Done, trial 65
========== MSC 13 252.34036450360088 ==========
Done, trial 66
========== MSC 14 254.6700105192906 ==========
Done, trial 67
========== MSC 15 256.9996565349803 ==========
Done, trial 68
========== MSC 16 259.32930255067 ==========
Done, trial 69
========